In [7]:
from datasets import Dataset
from transformers import AutoTokenizer
# conda env export > environment.yml
# conda env create -f environment.yml


LOAD DATA

In [8]:

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
# # Load data from files
# train_data = Dataset.from_file("../DATA/train.txt")
# label_data = Dataset.from_file("../DATA/label.txt")
# test_data = Dataset.from_file("../DATA/test.txt")
# val_data = Dataset.from_file("../DATA/valid.txt")
from datasets import load_dataset

dataset = load_dataset("tner/tweebank_ner")


# Preprocess the data
def preprocess(data):
    tokens = [item['tokens'] for item in data]
    tags = [item['tags'] for item in data]
    return {"tokens": tokens, "tags": tags}


In [9]:
train=dataset['train']
test=dataset['test']
val=dataset['validation']

In [10]:
train.shape,test.shape,val.shape

((1639, 2), (1201, 2), (710, 2))

In [11]:
train.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tags': Sequence(feature=ClassLabel(names=['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'], id=None), length=-1, id=None)}

we have to features tokens and tags.
token is text converted into id
tags is NER assigned having total 8 NE.

In [12]:
train['tags'][0],len(train['tags'][0])
#3for 1st text NE assigned 
#tags={"B-LOC": 0, "B-MISC": 1, "B-ORG": 2, "B-PER": 3, "I-LOC": 4, "I-MISC": 5, "I-ORG": 6, "I-PER": 7, "O": 8}

([8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], 18)

In [13]:
train['tokens'][0],len(train['tokens'][0])

(['RT',
  '@USER2362',
  ':',
  'Farmall',
  'Heart',
  'Of',
  'The',
  'Holidays',
  'Tabletop',
  'Christmas',
  'Tree',
  'With',
  'Lights',
  'And',
  'Motion',
  'URL1087',
  '#Holiday',
  '#Gifts'],
 18)

we can see the length of tags and tokens is same means each token assigned with NE.

## Preprocessing

In [25]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["tokens"], truncation=True, is_split_into_words=True,padding=True,return_tensors="pt")

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)
tokenized_val = val.map(tokenize_function, batched=True)


Map: 100%|██████████| 710/710 [00:00<00:00, 1613.52 examples/s]


In [27]:
tokenized_train.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tags': Sequence(feature=ClassLabel(names=['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'], id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

'tokens':

Type: Sequence of strings
Description: Represents the tokenized text sequences.
'tags':

Type: Sequence of labels
Description: Contains the labels representing named entity types associated with each token. The labels correspond to the following categories: 'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'.
'input_ids':

Type: Sequence of integers
Description: Holds the token IDs after tokenization, suitable for model input.
'token_type_ids':

Type: Sequence of integers
Description: Typically used in models like BERT for distinguishing different segments of the input.
'attention_mask':

Type: Sequence of integers
Description: Represents a binary mask indicating padding elements and actual tokens within the sequence.

In [28]:
tokenized_train.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tags': Sequence(feature=ClassLabel(names=['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'], id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [17]:
label2id = {
    "B-LOC": 0,
    "B-MISC": 1,
    "B-ORG": 2,
    "B-PER": 3,
    "I-LOC": 4,
    "I-MISC": 5,
    "I-ORG": 6,
    "I-PER": 7,
    "O": 8
}


In [29]:
tokenized_train

Dataset({
    features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1639
})

In [30]:
tokenized_train['tokens'][0:3]

[['RT',
  '@USER2362',
  ':',
  'Farmall',
  'Heart',
  'Of',
  'The',
  'Holidays',
  'Tabletop',
  'Christmas',
  'Tree',
  'With',
  'Lights',
  'And',
  'Motion',
  'URL1087',
  '#Holiday',
  '#Gifts'],
 ['#Volunteers',
  'are',
  'key',
  'members',
  'of',
  '#CHEO’s',
  'One',
  'Team',
  '–',
  'helping',
  'kids',
  'and',
  'families',
  'be',
  'their',
  'healthiest',
  '#NVW2016',
  'URL1387'],
 ['@USER2092',
  'is',
  "n't",
  'it',
  'funny',
  'how',
  'that',
  'always',
  'seems',
  'to',
  'work']]

In [42]:
# features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask']
def find_mismatched_lengths(examples):
    mismatched_indices = [i for i, (tokens, tags) in enumerate(zip(examples["tokens"], examples["tags"])) if len(tokens) != len(tags)]
    print(mismatched_indices)
    return mismatched_indices
non_m=find_mismatched_lengths(tokenized_train)
print(len(non_m))

[]
0


## Model Building

In [52]:
data={'tokens':train['tokens'][0:-1],
      'tags':train['tags'][0:-1]}

In [53]:
import pandas as pd
import numpy as np
df = pd.DataFrame(data)

In [54]:
df.head()

,tokens,tags
0,"[RT, @USER2362, :, Farmall, Heart, Of, The, Ho...","[8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
1,"[#Volunteers, are, key, members, of, #CHEO’s, ...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
2,"[@USER2092, is, n't, it, funny, how, that, alw...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"
3,"[RT, @USER80, :, Silence, is, better, than, li...","[8, 8, 8, 8, 8, 8, 8, 8, 8]"
4,"[I, just, spent, twenty, minutes, trying, to, ...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"


In [58]:
from transformers import pipeline
# create pipeline for NER
# ner = pipeline('ner', aggregation_strategy = 'simple')

In [59]:
# ner("Hi, my name is Ganesh Lokare. I am from Pune. I want to work with Google.")

In [61]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Tokenize the text
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# encoded_text = tokenizer(df['tokens'].tolist(), padding='max_length', truncation=True, max_length=512)

# Create the input_ids, attention_mask, and token_type_ids tensors
# input_ids = np.array(encoded_text['input_ids'])
# attention_mask = np.array(encoded_text['attention_mask'])
# token_type_ids = np.array(encoded_text['token_type_ids'])


In [65]:
df['tags']

0       [8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...
1       [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...
2                       [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
3                             [8, 8, 8, 8, 8, 8, 8, 8, 8]
4        [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
                              ...                        
1633                 [8, 8, 8, 8, 8, 8, 3, 8, 8, 8, 8, 8]
1634                 [8, 8, 8, 8, 3, 8, 1, 5, 8, 8, 8, 8]
1635                 [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
1636                          [8, 8, 8, 8, 8, 8, 8, 8, 8]
1637    [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...
Name: tags, Length: 1638, dtype: object

In [66]:
df['tokens']

0       [RT, @USER2362, :, Farmall, Heart, Of, The, Ho...
1       [#Volunteers, are, key, members, of, #CHEO’s, ...
2       [@USER2092, is, n't, it, funny, how, that, alw...
3       [RT, @USER80, :, Silence, is, better, than, li...
4       [I, just, spent, twenty, minutes, trying, to, ...
                              ...                        
1633    [RT, @USER1355, :, [, TIFFACHAT, ], stephaniey...
1634    [RT, @USER1701, :, FT, ISLAND, -, I, Hope, (, ...
1635    [@USER1321, @USER2526, Probably, ., He, is, n'...
1636    [RT, @USER1920, :, @USER1260, @USER2624, it, '...
1637    [You, have, that, right, ,, nor, do, they, int...
Name: tokens, Length: 1638, dtype: object

In [43]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Preparing the model
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id))

training_args = TrainingArguments(
    output_dir="./results",  # Directory to store model output
    num_train_epochs=3,       # Number of training epochs
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",     # Directory for logs
    logging_steps=100,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)



Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [44]:
# Train the model
trainer.train()

  0%|          | 0/309 [00:00<?, ?it/s]

ValueError: expected sequence of length 53 at dim 1 (got 61)